# Home Work 2 (Connection to camera (all 3 functionalities))

Capable of drawing with brush, rectangles and lines

Keys:

* q - Exit

* c - Clear from all the filter rectangles

* 0 - Change the filter to RGB2BGR

* 1 - Change the filter to RGB2HSV

* 2 - Change the filter to RGB2HLS

* 3 - Change the filter to blurring with ksize = [40, 40]

In [1]:
#HW Camera work (Projects 1, 2, 3)

import matplotlib.pyplot as plt
import cv2
import numpy as np
vid = cv2.VideoCapture(0)

ret, frame = vid.read()

x, y = frame.shape[0] // 2, frame.shape[1] // 2

width = int(vid.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
print(x, y, frame[:x, :y].shape)

%matplotlib inline

cv2.namedWindow("frame")

result = cv2.VideoWriter("cv_result.avi", cv2.VideoWriter_fourcc(*'XVID'), 60, (width, height))

mpos = None

prepos = None

released = True

myFilter = lambda x: np.array(map(max(255, x * 2), x))

filters = [
    lambda x: cv2.cvtColor(x, cv2.COLOR_RGB2BGR),
    lambda x: cv2.cvtColor(x, cv2.COLOR_RGB2HSV),
    lambda x: cv2.cvtColor(x, cv2.COLOR_RGB2HLS),
    lambda x: cv2.blur(x, [40, 40]),
]

filtChoice = 0

rects = [[[0, 100], [100, 200], 1]]

def mouseCallback(event, x, y, flags, params):
    global prepos, mpos, frame, ret, result, released, rects
    
    if flags == cv2.EVENT_FLAG_LBUTTON and released:
        mpos = None
        prepos = None
        released = False
    elif flags == cv2.EVENT_FLAG_LBUTTON and not prepos:
        prepos = [max(min(x, frame.shape[1]), 0), max(min(y, frame.shape[0]), 0)]
        mpos = [max(min(x, frame.shape[1]), 0), max(min(y, frame.shape[0]), 0)]
    elif flags == cv2.EVENT_FLAG_LBUTTON:
        mpos = [max(min(x, frame.shape[1]), 0), max(min(y, frame.shape[0]), 0)]
    elif flags != cv2.EVENT_FLAG_LBUTTON:
        if mpos and prepos:
            mnx, mny, mxx, mxy = min(prepos[0], mpos[0]), min(prepos[1], mpos[1]), max(prepos[0], mpos[0]), max(prepos[1], mpos[1])
            rects.append([[mnx, mny], [mxx, mxy], filtChoice])
        released = True

cv2.setMouseCallback("frame", mouseCallback)

while True:
    global frame, ret, mpos, prepos

    ret, preframe = vid.read()

    frame = preframe.copy()

    frame[:x, :y] = cv2.cvtColor(preframe[:x, :y], cv2.COLOR_RGB2BGR)
    frame[x:, :y] = cv2.blur(preframe[x:, :y], [40, 40])
    frame[:x, y:] = cv2.cvtColor(preframe[:x, y:], cv2.COLOR_RGB2HSV)

    for rect in rects:
        try:
            if rect[0][0] != rect[1][0] and rect[0][1] != rect[1][1]:
                frame[rect[0][1]:rect[1][1], rect[0][0]:rect[1][0]] = filters[rect[2]](preframe[rect[0][1]:rect[1][1], rect[0][0]:rect[1][0]])
        except:
            print(rect)
            print("LOL HERE")
    
    if mpos and prepos:
        mnx, mny, mxx, mxy = min(prepos[0], mpos[0]), min(prepos[1], mpos[1]), max(prepos[0], mpos[0]), max(prepos[1], mpos[1])
        if mnx != mxx and mxy != mny:
            frame[mny:mxy, mnx:mxx] = filters[filtChoice](preframe[mny:mxy, mnx:mxx])
            cv2.rectangle(frame,[mnx, mny], [mxx, mxy], (0, 255, 255), 3)
    
    k = cv2.waitKey(1)
#     frame[x:, y:] =

    result.write(frame)
    
    cv2.imshow("frame", frame)
    
    if k == ord('q'):
        break
        
    if k == ord('c'):
        mpos = None
        prepos = None
        rects = []
        
    if k in [ord('0') + i for i in range(4)]:
        filtChoice = k - ord('0')
        
result.release() 
vid.release()
cv2.destroyAllWindows()

240 320 (240, 320, 3)
